In [1]:
# mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Prep dataset

In [3]:
# prepare data
from datasets import load_dataset
eval_dataset = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem", split="train[:1%]")

README.md: 0.00B [00:00, ?B/s]

medical_o1_verifiable_problem.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40644 [00:00<?, ? examples/s]

In [4]:
eval_dataset

Dataset({
    features: ['Open-ended Verifiable Question', 'Ground-True Answer'],
    num_rows: 406
})

In [5]:
import pandas as pd
eval_dataset = pd.DataFrame(eval_dataset)
eval_dataset.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


In [6]:
# prompt_template = "You are an experienced medical professional. Answer the following medial question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: {}"

# eval_dataset['prompt'] = eval_dataset.apply(lambda x: prompt_template.format(x['Open-ended Verifiable Question']), axis=1)
# eval_dataset.prompt[0]

In [7]:
# from datasets import Dataset
# ds = Dataset.from_pandas(eval_dataset[['prompt']])
# ds

## Set up model definition

In [8]:
# with open('/content/drive/My Drive/w266/project/266 Final Project/llama3pt2-1b-peft/adapter_config.json', 'r') as f:
#   l = f.readlines()

In [9]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata

# Load the LoRA model
model_id = "erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-alt-prompt"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# # Load base model
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=quantization_config,
#     token=userdata.get('HF_TOKEN'))

In [10]:
# Load tokenizer
base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True,
                                          token=userdata.get('HF_TOKEN'))
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

1

In [11]:
tokenizer.get_chat_template()

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

## Eval of LoRA SFT (R = 8) model

In [12]:
# set up inference pipeline
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model_id,
                model_kwargs={
                    "quantization_config":quantization_config,
                },
                token=userdata.get('HF_TOKEN'))


adapter_config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

Device set to use cuda:0


In [13]:
fn_out = '/content/drive/MyDrive/w266_project/lora-sft-r8-output-reformatted.txt'
with open(fn_out, 'w') as f:
    f.write('\n')

In [14]:
for idx,sample in enumerate(eval_dataset['Open-ended Verifiable Question']):
  prompt = [
      {"role":"system","content":"You are a medical expert"},
      {"role":"user","content": "Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample}
  ]
  # prompt = tokenizer.apply_chat_template(prompt, tokenize=False)

  # prompt = "You are a medical expert. Provide a succinct answer to the following question, then rate the correctness of your answer as 1) Correct, 2) Incorrect, or 3) I Don't Know: "+sample
  for r in range(5):
    outputs = pipe(
        prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )
    response = outputs[0]["generated_text"]
    response = response.replace('\n','\t')
    print(response)
    with open(fn_out, 'a') as f:
        f.write(f"{idx}*****!!!!!"+response+'\n')

  if idx > 100:
    break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely cause of the gastrointestinal blood loss in this 88-year-old woman is non-steroidal anti-inflammatory drugs (NSAIDs) such as naproxen. This class of medications is known to contribute to gastric ulcers, which can lead to bleeding.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely cause of this woman's gastrointestinal blood loss, given her use of naproxen, is an erosive gastritis, particularly an anticoagulant-induced effect from the naproxen. This drug inhibits platelet activation and can exacerbate bleeding in individuals with similar conditions, such as peptic ulcers or those with anticoagulant therapy. Therefore, the correct explanation for her symptoms is an erosive gastritis due to the effects of naproxen on the gastric mucosa.		Rating: 1) Correct


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Based on the symptoms described, the most likely diagnosis for the 23-year-old male's condition is molluscum contagiosum.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely diagnosis for this condition is fixed condyloma. This condition aligns with the description provided, as it involves recurring scaly lesions that heal with slight hyperpigmentation and always appear at the same location.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely diagnosis for this condition is fixed condyloma acuminatum. This type of condyloma is characterized by recurrent lesions at the same location, which aligns with the symptoms described.
The most likely diagnosis for this condition is fixed balanitis, where the lesions are often caused by a bacterial infection. This is due to the characteristic presentation of scaly lesions on the glans that always appear at the same location and heal with slight hyperpigmentation, indicating a persistent infection at the site.		I rate the correctness of my answer as: 1. Correct.
